# Retrieve RSS

In [ ]:

!pip install feedparser requests importlib bs4 ffmpeg-python nltk soundfile numpy scipy firebase-admin google-cloud-firestore
!pip install git+https://github.com/suno-ai/bark.git


  Using cached google_api_python_client-2.151.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached PyJWT-2.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_core-2.22.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached google_cloud_firestore-2.19.0-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached msgpack-1.1.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
  Using cached googleapis_common_protos-1.65.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached proto_plus-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth-2.35.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached grpcio-1.67.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
  Using cached grpcio_status-1.67.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached u

In [1]:
from pyradiozilla import firebase

rz_firebase = firebase.Firebase()

In [2]:
from pyradiozilla import firebase

rz_author = firebase.RzAuthor(
    id= firebase.generate_url_safe_id('rss/weekly'),
    name='rss/weekly',
    description='Weekly rss summary scraper',        
    image=firebase.Blob(file_path="images/file-person.svg")
)
rz_author.upload_and_save(rz_firebase)
rz_channel = firebase.RzChannel(
    id= firebase.generate_url_safe_id('rss/python'),
    name='rss/python',    
    description='Latest updates about python programming language.',
    source_urls=['http://feeds.feedburner.com/PythonInsider'],
    image=firebase.Blob(file_path="images/rss.svg")
)
rz_channel.upload_and_save(rz_firebase)

In [3]:
from pyradiozilla import rss
from pyradiozilla import prompts
from pyradiozilla import ollama
from pyradiozilla import tts
from pyradiozilla import storage
import IPython
from pyradiozilla import firebase

directory = storage.Directory("/media/timur/TESLA SSD").get_timed_directory("run")
entries = rss.RssScraper('http://feeds.feedburner.com/PythonInsider').parse()
summaryConfug = prompts.SummaryConfig(
    "English",
    prompts.SummaryLength.medium,
    prompts.SummaryTone.neutral,
    [prompts.SummaryFocus.key_points],
)
rss_promots = prompts.RssWeeklySummaryPropmpt(entries, summaryConfug).get_prompts() 

ollama_client = ollama.OllamaClient("gemma2")
summaries = {}
for week, prompt in rss_promots.items():
    print(f"week: {week} prompt: {prompt}")
    directory.get_timed_file("prompt.txt").write_json(prompt)
    summary = ollama_client.generate(prompt)
    summaries[week] = summary
    audio = tts.BarkTTS.generate(summary, directory) 
    
    rz_audio = firebase.RzAudio(
        id= firebase.generate_url_safe_uuid(),
        name='rss/python/weekly',
        description='Weekly rss summary about python programming language.',
        author_id=rz_author.id,
        channel_id=rz_channel.id,
        image=firebase.Blob(file_path="images/rss.svg"),
        audio=firebase.Blob(file_path=audio.ogg_file_path),
        topics=['Python', 'Programming']
    )
    rz_audio.upload_and_save(rz_firebase)
        

IPython.display.display(summaries)

/home/timur/radiozilla/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


week: 2024-10-14 00:00:00 prompt: Provide a neutral summary in English that focuses on key points. The summary should be medium in length.

News from 2024-10-14 to 2024-10-20:

Python 3.14.0 alpha 1 is now available: It's now time for a new alpha of a new version of Python!
https://www.python.org/downloads/release/python-3140a1/
This is an early developer preview of Python
3.14
Major
new features of the 3.14 series, compared to 3.13
Python 3.14 is still in development. This release, 3.14.0a1 is the
first of seven planned alpha releases.
Alpha releases are intended to make it easier to test the current
state of new features and bug fixes and to test the release process.
During the alpha phase, features may be added up until the start of
the beta phase (2025-05-06) and, if necessary, may be modified or
deleted up until the release candidate phase (2025-07-22). Please keep
in mind that this is a preview release and its use is
not recommended for production environments.
Many new features 

/home/timur/radiozilla/.venv/lib/python3.12/site-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_locati

Conversion successful: /media/timur/TESLA SSD/2024-11-06T13.00.52.147191_run/2024-11-06T13.05.31.880590_audio.ogg
Object successfully written to /media/timur/TESLA SSD/2024-11-06T13.00.52.147191_run/2024-11-06T13.05.31.880590_audio.txt
week: 2024-10-07 00:00:00 prompt: Provide a neutral summary in English that focuses on key points. The summary should be medium in length.

News from 2024-10-07 to 2024-10-13:

Python 3.13.0 (final) released: Python 3.13.0 is now availablehttps://www.python.org/downloads/release/python-3130/This is the stable release of Python 3.13.0Python 3.13.0 is the newest major release of the Python programming language, and it contains many new features and optimizations compared to Python 3.12. (Compared to the last release candidate, 3.13.0rc3, 3.13.0 contains two small bug fixes and some documentation and testing changes.)Major new features of the 3.13 series, compared to 3.12Some of the new major new features and changes in Python 3.13 are:New featuresA new and

/home/timur/radiozilla/.venv/lib/python3.12/site-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():
100%|██████████| 29/29 [00:26<00:00,  1.11it/s]
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --en

...

/home/timur/radiozilla/.venv/lib/python3.12/site-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():
100%|██████████| 32/32 [00:27<00:00,  1.15it/s]
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --en

Conversion successful: /media/timur/TESLA SSD/2024-11-06T13.00.52.147191_run/2024-11-06T14.39.32.676572_audio.ogg
Object successfully written to /media/timur/TESLA SSD/2024-11-06T13.00.52.147191_run/2024-11-06T14.39.32.676572_audio.txt
week: 2023-10-02 00:00:00 prompt: Provide a neutral summary in English that focuses on key points. The summary should be medium in length.

News from 2023-10-02 to 2023-10-08:

Python 3.11.6 is now available: This is the sixth maintenance release of Python 3.11Python 3.11.6 is the newest major release of the Python programming language, and it contains many new features and optimizations. Get it here:https://www.python.org/downloads/release/python-3116/Major new features of the 3.11 series, compared to 3.10Among the new major new features and changes so far:PEP 657 – Include Fine-Grained Error Locations in TracebacksPEP 654 – Exception Groups and except*PEP 673 – Self TypePEP 646 – Variadic GenericsPEP 680 – tomllib: Support for Parsing TOML in the Stand

/home/timur/radiozilla/.venv/lib/python3.12/site-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():
100%|██████████| 28/28 [00:23<00:00,  1.19it/s]
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --en

Conversion successful: /media/timur/TESLA SSD/2024-11-06T13.00.52.147191_run/2024-11-06T14.47.00.287688_audio.ogg
Object successfully written to /media/timur/TESLA SSD/2024-11-06T13.00.52.147191_run/2024-11-06T14.47.00.287688_audio.txt


{datetime.datetime(2024, 10, 14, 0, 0): 'The Python Software Foundation has released the first alpha version of Python 3.14.0. This is an early preview for developers to test new features and bug fixes before the official release in late 2025.\n\nKey highlights of this alpha release include:\n\n* **Deferred evaluation of annotations (PEP 649):** This improves type checking performance.\n* **Improved error messages:**  Making it easier to understand and debug errors.\n\nThe development team is actively working on more features for Python 3.14, which will be announced closer to the beta release in May 2025. Alpha releases are not recommended for production use due to potential instability. Users can report bugs and contribute to the project through the official channels provided in the announcement.',
 datetime.datetime(2024, 10, 7, 0, 0): "Python 3.13.0 has been officially released, bringing several significant improvements and new features compared to its predecessor, Python 3.12.  \n\